<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Generative-Adversarial-Network/blob/main/TF2_0_Deep_Convolutional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout, Dense, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [ ]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels) #1차원 흑백이미지 28x28
z_dim = 100 # Latent space 


In [ ]:
#Designing Generator

def build_generator(z_dim) : 
  model = Sequential()
  model.add(Dense(512*7*7, input_dim = z_dim))
  model.add(Reshape((7,7,512)))

  model.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.02))
  model.add(Conv2DTranspose(64, kernel_size = 3, strides = 1, padding = 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.02))
  model.add(Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(Activation('tanh'))

  return model

In [ ]:
#Designing Discriminator

def build_discriminator(img_shape) : 
  model = Sequential()
  model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = img_shape, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.02))
  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.02))
  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.02))
  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))

  return model

In [ ]:
def build_gan(generator, discriminator) : 
  model = Sequential()
  model.add(generator)
  model.add(discriminator)

  return model

In [ ]:
discriminator = build_discriminator(img_shape)
discriminator.compile(loss = 'binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

generator = build_generator(z_dim)
discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(loss = 'binary_crossentropy', optimizer = Adam())

In [ ]:
losses = []
accuracies = []
iteration_checkpoints = []


def train(iterations, batch_size, sample_interval):

    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train / 127.5 - 1.0
    X_train = np.expand_dims(X_train, axis=3)

    real = np.ones((batch_size, 1))

    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        d_loss_real = discriminator.train_on_batch(imgs, real) #Batch갯수 전체에 대해 각각 1/0으로 Output나오는걸로 학습
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)#Random Gen한건 0으로 인식하도록 Discriminator 학습
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z) #Noise image (Basic Latent Space)

        g_loss = gan.train_on_batch(z, real) #latent space로 Gen한것을 Real로 인식하도록 Generator 학습

        if (iteration + 1) % sample_interval == 0:

            # 훈련이 끝난 후 그래프를 그리기 위해 손실과 정확도 저장
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # 훈련 과정 출력
            print("%d [Discriminator Loss: %f, Accuracy: %.2f%%] [Generator Loss: %f]" %
                  (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

            # 생성된 이미지 샘플 출력
            sample_images(generator)

In [ ]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):

    # 랜덤한 잡음 샘플링
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # 랜덤한 잡음에서 이미지 생성하기
    gen_imgs = generator.predict(z)

    # 이미지 픽셀 값을 [0, 1] 사이로 스케일 조정
    gen_imgs = 0.5 * gen_imgs + 0.5

    # 이미지 그리드 설정
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # 이미지 그리드 출력
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1

In [ ]:
# 하이퍼파라미터 설정
iterations = 20000
batch_size = 128
sample_interval = 1000

# 지정된 반복 횟수만큼 GAN 훈련
train(iterations, batch_size, sample_interval)